# NASA Exoplanet Mass Prediction

This notebook predict Mass of a exoplanet using NASA Exoplanet Archive data

# Import Libraries and Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from scipy.stats import jarque_bera, normaltest, shapiro, chi2_contingency
from statsmodels.stats.diagnostic import het_white, het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer, QuantileTransformer, PolynomialFeatures, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression, RFE, RFECV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.inspection import permutation_importance
import itertools
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
np.random.seed(42)

# Data Loading and Initial Inspection

In [2]:
url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query=select+*+from+ps&format=csv"
df = pd.read_csv(url)

print(f"Dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\nColumn Data Types:")
print(df.dtypes.value_counts())

target_col = 'pl_bmasse'
print(f"\nTarget variable '{target_col}' statistics:")
print(df[target_col].describe())

Dataset shape: (38898, 354)
Memory usage: 291.88 MB

Column Data Types:
float64    239
object      89
int64       26
Name: count, dtype: int64

Target variable 'pl_bmasse' statistics:
count     6866.000000
mean       710.053607
std       1451.794928
min          0.015000
25%         13.300000
50%        190.000000
75%        664.979818
max      25426.400000
Name: pl_bmasse, dtype: float64


# Data Quality Assessment and Missing Value Analysis

In [3]:
missing_stats = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
    'Data_Type': df.dtypes
})
missing_stats = missing_stats[missing_stats['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

relevant_features = ['pl_bmasse', 'pl_rade', 'pl_orbper', 'pl_orbsmax', 'pl_eqt',
                    'st_mass', 'st_rad', 'st_teff', 'st_met', 'st_age', 'st_dens',
                    'pl_dens', 'discoverymethod', 'disc_year', 'sy_dist']

df_filtered = df[relevant_features].copy()

df_clean = df_filtered.dropna(subset=[target_col])

numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()

for col in numeric_cols:
    if col != target_col:
        df_clean[col].fillna(df_clean[col].median(), inplace=True)

for col in categorical_cols:
    df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)

print(f"Clean dataset shape: {df_clean.shape}")
print(f"Remaining missing values: {df_clean.isnull().sum().sum()}")

Clean dataset shape: (6866, 15)
Remaining missing values: 0
